# FeedBack Prize: EDA

## Requirments:

In [2]:
!pip install pandas

     |████████████████████████████████| 12.2 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 500 kB 48.3 MB/s eta 0:00:01


## Libraries

In [3]:
import pandas as pd

## Configs

In [4]:
class CFG:
    # File paths:
    common_path = '/srv/dataset/datafbp'
    train_path = f'{common_path}/train.csv'
    test_path = f'{common_path}/test.csv'
    sub_path = f'{common_path}/sample_submission.csv'

## Load dataframe

In [6]:
train_df = pd.read_csv(CFG.train_path)
test_df = pd.read_csv(CFG.test_path)

In [7]:
train_df

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5
